In [11]:
# Shaurya Jeloka, Akshay Vakharia, Ian Jeong
# 6/9/2024

# This program creates an interactive investment dashboard using IPython, ipywidgets, and Dash, allowing 
# users to track prices of specified Amazon items, stocks, and cryptocurrencies, and manage their portfolios. 
# This program also displays current lists of Amazon items, stocks, and cryptocurrencies being tracked,
# gives users editable text areas to update the lists of items being tracked, creates a Dash app with 
# tabs for stock analysis, crypto analysis, and portfolio management, calculates and shows various financial 
# indicators for stocks and cryptocurrencies including moving averages, bollinger bands, daily returns, and 
# cumulative returns, and shows users metrics for stock and crypto portfolios, including total value, total 
# investment, and profit/loss. This is the only file external users will need to open and interact with and
# it is designed to be easy to use and aesthetic.

In [12]:
import pandas as pd
import time
import numpy as np
import sqlite3
import json

import IPython
from IPython.display import display, SVG, clear_output, HTML
from ipywidgets import widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

import numpy as np

In [18]:
# Create connection object to database with price history
con = sqlite3.connect("prices.db")

# Store the pricing history for stocks, amazon items, and cryptocurrencies from
# the database as dataframes
stocks = pd.read_sql("SELECT * FROM stocks;", con)
amazon_items = pd.read_sql("SELECT * FROM amazon_items;", con)
cryptocurrencies = pd.read_sql("SELECT * FROM cryptocurrencies;", con)

# Define trackitems.json as the file name to store info on user's portfolio and
# items to track
DATA_FILE = "trackitems.json"

# Close the connection object
con.close()

In [19]:
# Define function to read data from JSON file
def read_data(filename):

    # Return json data in file if file exists
    try:
        with open(filename, "r") as file:
            return json.load(file)

    # In case the file cannot be found, return empty lists for each item meant to be stored
    except FileNotFoundError:
        return {"amazon_items": [], "stock_tickers": [], "crypto_names": "", "portfolio": [], 
                "crypto_portfolio": []}


# Define function to write data to JSON file
def write_data(filename, data):
    with open(filename, "w") as file:
        json.dump(data, file, indent=4)


# Read data from json file and store the data into separate variables
data = read_data(DATA_FILE)
amazon_item_list = data["amazon_items"]
stock_tickers = data["stocks"]
crypto_names = data["cryptocurrencies"]
portfolio = data["portfolio"]
crypto_portfolio = data["crypto_portfolio"]

In [15]:
# Create HTML label for Amazon items
amazon_label = widgets.HTML(value="<b>Current Amazon Items:</b>")

# Create text area to display currently tracked amazon items
amazon_current = widgets.Textarea(
    value="\n".join(amazon_item_list), disabled=True, 
    layout=widgets.Layout(width="100%", height="100px")
)

# Create HTML label for stocks
stocks_label = widgets.HTML(value="<b>Current Stocks:</b>")

# Create text area to display currently tracked stocks
stocks_current = widgets.Textarea(
    value="\n".join(stock_tickers), disabled=True, layout=widgets.Layout(width="100%", height="100px")
)

# Create HTML label for cryptocurrencies
cryptos_label = widgets.HTML(value="<b>Current Cryptocurrencies:</b>")

# Create text area to display currently tracked cryptocurrencies
cryptos_current = widgets.Textarea(
    value=crypto_names, disabled=True, layout=widgets.Layout(width="100%", height="100px")
)

# Create text areas for editing each of the currently tracked items
amazon_edit = widgets.Textarea(
    value="\n".join(amazon_item_list),
    placeholder="Add Amazon items",
    description="Edit:",
    layout=widgets.Layout(width="100%", height="150px"),
)
stocks_edit = widgets.Textarea(
    value="\n".join(stock_tickers),
    placeholder="Add stocks",
    description="Edit:",
    layout=widgets.Layout(width="100%", height="150px"),
)
cryptos_edit = widgets.Textarea(
    value=crypto_names,
    placeholder="Add cryptocurrencies separated by commas",
    description="Edit:",
    layout=widgets.Layout(width="100%", height="150px"),
)

# Create button to update the lists
update_button = widgets.Button(
    description="Update Lists", button_style="success", layout=widgets.Layout(width="30%", height="40px")
)

# Create output area
output = widgets.Output()


# Define function to update the lists of currently tracked items
def update_lists(b):

    # Update the dictionaries with user edited values
    data["amazon_items"] = amazon_edit.value.split("\n")
    data["stock_tickers"] = stocks_edit.value.split("\n")
    data["crypto_names"] = cryptos_edit.value

    # Write updated data to the json file
    write_data(DATA_FILE, data)

    # Update the current displays with new values
    amazon_current.value = "\n".join(data["amazon_items"])
    stocks_current.value = "\n".join(data["stock_tickers"])
    cryptos_current.value = data["crypto_names"]

    # Print success message
    with output:
        clear_output()
        print("Lists updated successfully!")


# Call update_lists function when update button is clicked
update_button.on_click(update_lists)

# Organize all the widgets in tabs to make UI more clean and navigable
tab = widgets.Tab()
tab.children = [
    widgets.VBox([amazon_label, amazon_current, amazon_edit, update_button]),
    widgets.VBox([stocks_label, stocks_current, stocks_edit, update_button]),
    widgets.VBox([cryptos_label, cryptos_current, cryptos_edit, update_button]),
]

#Set titles for tabs
tab.set_title(0, "Amazon Items")
tab.set_title(1, "Stocks")
tab.set_title(2, "Cryptocurrencies")

# Display all widgets in a vertical box
ui = widgets.VBox([tab, output])
display(ui)

In [17]:
# Helper function to convert timestamps and sort data
def preprocess_data(df, group_column):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values(by=[group_column, "timestamp"])
    return df


# Helper function to calculate indicators
def calculate_indicators(df, group_column):

    # Calculate the 20-day moving average
    df["20_MA"] = df.groupby(group_column)["price"].transform(
        lambda x: x.rolling(window=20, min_periods=1).mean())

     # Calculate the 20-day standard deviation
    df["20_STD"] = df.groupby(group_column)["price"].transform(
        lambda x: x.rolling(window=20, min_periods=1).std())

    # Calculate upper and lower Bollinger bands & average and cumulative returns
    df["Upper_BB"] = df["20_MA"] + (df["20_STD"] * 2)
    df["Lower_BB"] = df["20_MA"] - (df["20_STD"] * 2)
    df["daily_return"] = df.groupby(group_column)["price"].pct_change()
    df["cumulative_return"] = (1 + df["daily_return"]).cumprod()
    return df


# Preprocess and calculate indicators for stocks and cryptocurrencies
stocks = preprocess_data(stocks, "stock_symbol")
cryptocurrencies = preprocess_data(cryptocurrencies, "crypto_name")

stocks = calculate_indicators(stocks, "stock_symbol")
cryptocurrencies = calculate_indicators(cryptocurrencies, "crypto_name")

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Investment Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Stock Analysis', children=[
            
            # Creates dropdown to select stock
            dcc.Dropdown(
                id='stock-dropdown',
                options=[{'label': stock, 'value': stock} for stock in stocks['stock_symbol'].unique()],
                value=stocks['stock_symbol'].unique()[0]
            ),
            
            # Creates checklist for moving average options, bollinger bands, etc.  
            dcc.Checklist(
                id='toggle-options',
                options=[
                    {'label': 'Show Moving Average', 'value': 'show_ma'},
                    {'label': 'Show Bollinger Bands', 'value': 'show_bb'},
                    {'label': 'Show Trend Line', 'value': 'show_trend'}
                ],
                value=['show_ma', 'show_bb', 'show_trend'],
                labelStyle={'display': 'inline-block'}
            ),

            # Generates graph to display stock price
            dcc.Graph(id='price-graph'),
            html.Div([
                
                # Creates date picker for stock data
                dcc.DatePickerSingle(
                    id='investment-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date(),
                    date=stocks['timestamp'].min().date()
                ),
                html.Div(id='investment-value')
            ]),
            html.Div([
                
                # Creates another date picker to select data for lowest price
                dcc.DatePickerSingle(
                    id='lowest-price-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date(),
                    date=stocks['timestamp'].min().date()
                ),
                html.Div(id='lowest-price')
            ]),
            dcc.Graph(id='correlation-matrix'),
            dcc.Graph(id='daily-returns-graph'),
            dcc.Graph(id='cumulative-returns-graph')
        ]),
        
        # Creates a tab for crypto analysis 
        dcc.Tab(label='Crypto Analysis', children=[
            
            # Creates a dropdown to select a cryptocurrency
            dcc.Dropdown(
                id='crypto-dropdown',
                # Creates options in dropdown from list of cryptocurrencies 
                options=[{'label': crypto, 'value': crypto} for crypto in cryptocurrencies['crypto_name'].unique()],
                value=cryptocurrencies['crypto_name'].unique()[0]
            ),

            # Creates a checklist similar to the one above for MA, Bollinger Bands, etc. 
            dcc.Checklist(
                id='crypto-toggle-options',
                options=[
                    {'label': 'Show Moving Average', 'value': 'show_ma'},
                    {'label': 'Show Bollinger Bands', 'value': 'show_bb'},
                    {'label': 'Show Trend Line', 'value': 'show_trend'}
                ],
                value=['show_ma', 'show_bb', 'show_trend'],
                labelStyle={'display': 'inline-block'}
            ),
            
            # Creates a graph to display cryptocurrency price data
            dcc.Graph(id='crypto-price-graph'),
            html.Div([

                # Creates date picker to select the investment data from a certain time
                dcc.DatePickerSingle(
                    id='crypto-investment-date-picker',

                    # Minimum & maximum date for when a user is allowed to put in their investing info
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date(),
                    date=cryptocurrencies['timestamp'].min().date()
                ),
                html.Div(id='crypto-investment-value')
            ]),
            html.Div([

                # Creates date picker to select date for lowest crypto price check
                dcc.DatePickerSingle(
                    id='crypto-lowest-price-date-picker',
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date(),
                    date=cryptocurrencies['timestamp'].min().date()
                ),
                html.Div(id='crypto-lowest-price')
            ]),
            dcc.Graph(id='crypto-correlation-matrix'),
            dcc.Graph(id='crypto-daily-returns-graph'),
            dcc.Graph(id='crypto-cumulative-returns-graph')
        ]),

        # Creates another tab for Portfolio Management
        dcc.Tab(label='Portfolio Management', children=[
            html.H3("Manage Portfolio"),
            
            # Generates an input field to add a stock to user's portfolio
            dcc.Input(
                id='portfolio-stock-input',
                type='text',
                placeholder='Add stock to portfolio',
                debounce=True
            ),
            html.Div([
                
                # Another date picker to select purchase date
                dcc.DatePickerSingle(
                    id='portfolio-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date()
                ),

                # Creates an input field to enter quantity
                dcc.Input(
                    id='portfolio-quantity-input',
                    type='number',
                    placeholder='Quantity',
                    min=1,
                ),

                # Creates an input field to enter price
                dcc.Input(
                    id='portfolio-price-input',
                    type='number',
                    placeholder='Price',
                    min=0.01,
                    step=0.01,
                )
            ]),

            # Creates buttons to add or remove stocks from the portfolio, respectively
            html.Button('Add to Portfolio', id='add-to-portfolio-button', n_clicks=0),
            html.Button('Remove Selected Stock', id='remove-from-portfolio-button', n_clicks=0),

            # Creates Dropdown object to select a stock for removal from the portfolio
            dcc.Dropdown(
                id='portfolio-remove-dropdown',
                options=[{'label': stock['symbol'], 'value': stock['symbol']} for stock in portfolio],
                placeholder='Select stock to remove'
            ),
            html.H3("Portfolio"),
            
            # Creates Div object to display the portfolio
            html.Div(id='portfolio-display'),
            html.H3("Portfolio Metrics"),

            # Creates Div object to display portfolio metrics
            html.Div(id='portfolio-metrics'),

            # Creates graph to display stock distribution of the portfolio
            dcc.Graph(id='stock-distribution'),
            html.H3("Manage Crypto Portfolio"),

            # Creates input field to add cryptocurrency to portfolio
            dcc.Input(
                id='portfolio-crypto-input',
                type='text',
                placeholder='Add crypto to portfolio',
                debounce=True
            ),
            html.Div([
                # Creates date picker to select purchase date for crypto
                dcc.DatePickerSingle(
                    id='crypto-portfolio-date-picker',
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date()
                ),

                # Creates input fields to enter the quantity for crypto 
                dcc.Input(
                    id='crypto-portfolio-quantity-input',
                    type='number',
                    placeholder='Quantity',
                    min=1,
                ),

                # Creates input fields to enter in price for crypto
                dcc.Input(
                    id='crypto-portfolio-price-input',
                    type='number',
                    placeholder='Price',
                    min=0.01,
                    step=0.01,
                )
            ]),

            # Creates buttons to add or remove cryptocurrencies from the portfolio
            html.Button('Add to Crypto Portfolio', id='add-to-crypto-portfolio-button', n_clicks=0),
            html.Button('Remove Selected Crypto', id='remove-from-crypto-portfolio-button', n_clicks=0),

            # Creates dropdown to select a crypto for removal
            dcc.Dropdown(
                id='crypto-portfolio-remove-dropdown',
                options=[{'label': crypto['symbol'], 'value': crypto['symbol']} for crypto in crypto_portfolio],
                placeholder='Select crypto to remove'
            ),
            html.H3("Crypto Portfolio"),

            # Final Div object to display portfolio
            html.Div(id='crypto-portfolio-display'),
            html.H3("Crypto Portfolio Metrics"),
            html.Div(id='crypto-portfolio-metrics'),
            dcc.Graph(id='crypto-distribution')
        ])
    ])
])

# Common callback functions for stocks and cryptocurrencies

# Define function that updates the graphs for stocks and cryptocurrencies
def update_graph(
    df, selected_item, toggle_options, group_column, ma_column, upper_bb_column, lower_bb_column):

    # filters data to the selected item
    item_data = df[df[group_column] == selected_item]
    traces = []

    # adds trace for price 
    price_trace = go.Scatter(
        x=item_data["timestamp"], y=item_data["price"], mode="lines+markers", name="Price")
    traces.append(price_trace)

    # Show moving average if selected
    if "show_ma" in toggle_options:
        ma_trace = go.Scatter(
            x=item_data["timestamp"],
            y=item_data[ma_column],
            mode="lines",
            name="20-Day MA",
            line=dict(width=1))
        traces.append(ma_trace)

    # show Bollinger bands if selected
    if "show_bb" in toggle_options:
        # Upper bollinger band
        upper_bb_trace = go.Scatter(
            x=item_data["timestamp"],
            y=item_data[upper_bb_column],
            mode="lines",
            name="Upper Bollinger Band",
            line=dict(color="rgba(0,100,80,0.2)", width=1))

        # Lower bollinger band
        lower_bb_trace = go.Scatter(
            x=item_data["timestamp"],
            y=item_data[lower_bb_column],
            mode="lines",
            name="Lower Bollinger Band",
            line=dict(color="rgba(0,100,80,0.2)", width=1),
            fill="tonexty")

        # adds bands
        traces.extend([upper_bb_trace, lower_bb_trace])

    # Show trend line if selected
    if "show_trend" in toggle_options:
        x_values = np.arange(len(item_data))
        y_values = item_data["price"].values
        if len(x_values) > 1:
            with np.warnings.catch_warnings():
                np.warnings.simplefilter("ignore", np.RankWarning)
                coeffs = np.polyfit(x_values, y_values, 1)
                polynomial = np.poly1d(coeffs)
                trendline = polynomial(x_values)

            # creates and appends trend line
            trend_trace = go.Scatter(
                x=item_data["timestamp"], y=trendline, mode="lines", name="Trend Line", line=dict(width=1))
            traces.append(trend_trace)

    # Defines final layout for the graph
    layout = go.Layout(
        title=f"Prices Over Time for {selected_item}",
        xaxis={"title": "Timestamp"},
        yaxis={"title": "Price"},
        legend={"x": 0, "y": 1},
        hovermode="closest")

    return {"data": traces, "layout": layout}


# Callback for stock and crypto graphs
@app.callback(
    [Output("price-graph", "figure"), Output("crypto-price-graph", "figure")], 
    [Input("stock-dropdown", "value"),Input("crypto-dropdown", "value"),
        Input("toggle-options", "value"), Input("crypto-toggle-options", "value")])

# Define function to update graphs
def update_graphs(selected_stock, selected_crypto, toggle_options, crypto_toggle_options):
    
    # Updates the stock graph with the selected options
    stock_figure = update_graph(
        stocks, selected_stock, toggle_options, "stock_symbol", "20_MA", "Upper_BB", "Lower_BB")

    # Updates the crypto graph with the selected options
    crypto_figure = update_graph(
        cryptocurrencies,
        selected_crypto,
        crypto_toggle_options,
        "crypto_name",
        "20_MA",
        "Upper_BB",
        "Lower_BB")
    return stock_figure, crypto_figure


# Combined callback for updating investment values
@app.callback(
    [Output("investment-value", "children"), Output("crypto-investment-value", "children")],
    [Input("stock-dropdown", "value"), Input("crypto-dropdown", "value"),
    Input("investment-date-picker", "date"), Input("crypto-investment-date-picker", "date")])

# Define function to update values of investments
def update_investment_values(selected_stock, selected_crypto, investment_date, crypto_investment_date):
    # Convert selected dates to datetime format 
    investment_date = pd.to_datetime(investment_date)
    crypto_investment_date = pd.to_datetime(crypto_investment_date)

    # Filters data for the selected stock AND for the selected investment date
    stock_data = stocks[
        (stocks["stock_symbol"] == selected_stock) & (stocks["timestamp"].dt.date == investment_date.date())]

    # Filters data for the selected crypto AND for the selected investment date
    crypto_data = cryptocurrencies[(cryptocurrencies["crypto_name"] == selected_crypto)
        & (cryptocurrencies["timestamp"].dt.date == crypto_investment_date.date())]

    # Calculates the current value of the stock investment 
    if stock_data.empty:

        # If no data available, states that
        stock_value = "No data available for the selected date."
    else:

        # Calculates initial and current price, multiplies by 1000 to get value of 1000$ investment
        initial_price = stock_data.iloc[0]["price"]
        current_price = stocks[stocks["stock_symbol"] == selected_stock].iloc[-1]["price"]
        investment_value = 1000 * (current_price / initial_price)
        stock_value = f"The current value of a $1000 investment in {selected_stock} on {investment_date.date()} is ${investment_value:.2f}."

    # Calculates the current value of the crypto investment
    if crypto_data.empty:

        # If no data available, states that
        crypto_value = "No data available for the selected date."
    else:

        # Calculates initial and current price, multiplies by 1000 to get value of 1000$ investment
        initial_price = crypto_data.iloc[0]["price"]
        current_price = cryptocurrencies[cryptocurrencies["crypto_name"] == selected_crypto].iloc[-1]["price"]
        investment_value = 1000 * (current_price / initial_price)
        crypto_value = f"The current value of a $1000 investment in {selected_crypto} on {crypto_investment_date.date()} is ${investment_value:.2f}."

    return stock_value, crypto_value


# Combined callback to display lowest prices since a given date
@app.callback(
    [Output("lowest-price", "children"), Output("crypto-lowest-price", "children")],
    [Input("stock-dropdown", "value"),Input("crypto-dropdown", "value"),
    Input("lowest-price-date-picker", "date"), Input("crypto-lowest-price-date-picker", "date")])

# Define a function to update the lowest prices
def update_lowest_prices(selected_stock, selected_crypto, since_date, crypto_since_date):
    
    # Convert selected dates to datetime format
    since_date = pd.to_datetime(since_date)
    crypto_since_date = pd.to_datetime(crypto_since_date)

    # Filters data for the selected stock since the given date
    stock_data = stocks[(stocks["stock_symbol"] == selected_stock) & (stocks["timestamp"] >= since_date)]
    
    # Filters data for the selected crypto since the given date
    crypto_data = cryptocurrencies[(cryptocurrencies["crypto_name"] == selected_crypto)
        & (cryptocurrencies["timestamp"] >= crypto_since_date)]

    # Determines the lowest price for the selected stock data since the given date
    if stock_data.empty:
        lowest_stock_price = f"No data available for {selected_stock} since {since_date.date()}."
    else:
        lowest_price = stock_data["price"].min()
        lowest_stock_price = f"The lowest recorded price for {selected_stock} since {since_date.date()} is ${lowest_price:.2f}."

    # Determines the lowest price for the selected crypto data since the given date
    if crypto_data.empty:
        lowest_crypto_price = f"No data available for {selected_crypto} since {crypto_since_date.date()}."
    else:
        lowest_price = crypto_data["price"].min()
        lowest_crypto_price = f"The lowest recorded price for {selected_crypto} since {crypto_since_date.date()} is ${lowest_price:.2f}."

    return lowest_stock_price, lowest_crypto_price


# Callback to update the correlation matrix
@app.callback(
    [Output("correlation-matrix", "figure"), Output("crypto-correlation-matrix", "figure")],
    [Input("stock-dropdown", "value"), Input("crypto-dropdown", "value")])

# Define function to update correlation matrices
def update_correlation_matrices(selected_stock, selected_crypto):
    
    # Pivots the data for stocks and crypto to create a correlation matrix
    stock_pivot_df = stocks.pivot(index="timestamp", columns="stock_symbol", values="price")
    crypto_pivot_df = cryptocurrencies.pivot(index="timestamp", columns="crypto_name", values="price")

    # Calculates the correlation matrix
    stock_correlation_matrix = stock_pivot_df.corr()
    crypto_correlation_matrix = crypto_pivot_df.corr()

    # Shows the correlation matrix for stocks 
    stock_fig = px.imshow(
        stock_correlation_matrix,
        x=stock_correlation_matrix.columns,
        y=stock_correlation_matrix.columns,
        color_continuous_scale="RdBu_r",
        zmin=-1,
        zmax=1)
    
    stock_fig.update_layout(title="Correlation Matrix of Stock Prices")

    # Shows the correlation matrix for crypto 
    crypto_fig = px.imshow(
        crypto_correlation_matrix,
        x=crypto_correlation_matrix.columns,
        y=crypto_correlation_matrix.columns,
        color_continuous_scale="RdBu_r",
        zmin=-1,
        zmax=1)
    
    crypto_fig.update_layout(title="Correlation Matrix of Crypto Prices")
    return stock_fig, crypto_fig


# Callback to update daily and cumulative returns graphs
@app.callback(
    [Output("daily-returns-graph", "figure"), Output("crypto-daily-returns-graph", "figure"),
    Output("cumulative-returns-graph", "figure"), Output("crypto-cumulative-returns-graph", "figure")],
    [Input("stock-dropdown", "value"), Input("crypto-dropdown", "value")])

# Define function to update graphs
def update_returns_graphs(selected_stock, selected_crypto):
    # Filters data to selected stock and crypto
    stock_data = stocks[stocks["stock_symbol"] == selected_stock]
    crypto_data = cryptocurrencies[cryptocurrencies["crypto_name"] == selected_crypto]

    # Creates a line plot for daily returns of stock data 
    daily_returns_fig = px.line(
        stock_data, x="timestamp", y="daily_return", title=f"Daily Returns for {selected_stock}")
    daily_returns_fig.update_layout(yaxis_title="Daily Return", xaxis_title="Timestamp")

    # Creates a line plot for daily returns of crypto data 
    crypto_daily_returns_fig = px.line(
        crypto_data, x="timestamp", y="daily_return", title=f"Daily Returns for {selected_crypto}")
    crypto_daily_returns_fig.update_layout(yaxis_title="Daily Return", xaxis_title="Timestamp")

    # Creates a line plot for cumulative returns of stock data
    cumulative_returns_fig = px.line(
        stock_data, x="timestamp", y="cumulative_return", title=f"Cumulative Returns for {selected_stock}")
    cumulative_returns_fig.update_layout(yaxis_title="Cumulative Return", xaxis_title="Timestamp")

    # Creates a line plot for cumulative returns of crypto data
    crypto_cumulative_returns_fig = px.line(
        crypto_data, x="timestamp", y="cumulative_return", title=f"Cumulative Returns for {selected_crypto}")
    crypto_cumulative_returns_fig.update_layout(yaxis_title="Cumulative Return", xaxis_title="Timestamp")

    return daily_returns_fig, crypto_daily_returns_fig, cumulative_returns_fig, crypto_cumulative_returns_fig


# Combined callback to manage stock portfolio and display metrics
@app.callback(
    [Output("portfolio-display", "children"), Output("portfolio-metrics", "children"),
    Output("portfolio-remove-dropdown", "options"), Output("stock-distribution", "figure")],
    [Input("add-to-portfolio-button", "n_clicks"), Input("remove-from-portfolio-button", "n_clicks")],
    [State("portfolio-stock-input", "value"), State("portfolio-date-picker", "date"),
    State("portfolio-quantity-input", "value"), State("portfolio-price-input", "value"),
    State("portfolio-remove-dropdown", "value")])

# Define function that allows user to manage/edit stock portfolio
def manage_portfolio(add_clicks, remove_clicks, stock_symbol, purchase_date, quantity, price, remove_symbol):
    ctx = dash.callback_context
    if not ctx.triggered:
        return display_portfolio_metrics(portfolio)

    button_id = ctx.triggered[0]["prop_id"].split(".")[0]

    # Adds new stock to portfolio 
    if button_id == "add-to-portfolio-button" and stock_symbol and purchase_date and quantity and price:
        new_entry = {"symbol": stock_symbol, "date": purchase_date, "quantity": quantity, "price": price}
        portfolio.append(new_entry)

    # Removes a selected stock from portfolio
    elif button_id == "remove-from-portfolio-button" and remove_symbol:
        portfolio[:] = [stock for stock in portfolio if stock["symbol"] != remove_symbol]

    data["portfolio"] = portfolio
    write_data(DATA_FILE, data)

    return display_portfolio_metrics(portfolio)

# Define function to display stock portfolio metrics
def display_portfolio_metrics(portfolio):

    # Creates list of portfolio items 
    portfolio_list = [
        f"{stock['symbol']} - {stock['quantity']} shares bought on {stock['date']} at ${stock['price']}"
        for stock in portfolio]

    total_value = 0
    total_quantity = 0
    total_investment = 0
    stock_metrics = []

    # Calculates total value, investment, profit/loss for portfolio
    for stock in portfolio:
        stock_data = stocks[stocks["stock_symbol"] == stock["symbol"]]
        if not stock_data.empty:
            current_price = stock_data["price"].iloc[-1]

            # Total investment value
            investment_value = stock["quantity"] * current_price
            total_investment += stock["quantity"] * stock["price"]

            # Profit Loss
            profit_loss = investment_value - (stock["quantity"] * stock["price"])
            total_value += investment_value
            total_quantity += stock["quantity"]
            
            # Appends values 
            stock_metrics.append(
                f"""{stock['symbol']}: {stock['quantity']} shares, Bought at ${stock['price']},\ 
                Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, P/L: ${profit_loss:.2f}"""
            )

    # Create a list of portfolio metrics
    portfolio_metrics = html.Ul(
        [html.Li(f"Total Portfolio Value: ${total_value:.2f}"),
        html.Li(f"Total Shares Owned: {total_quantity}"),
        html.Li(f"Total Investment: ${total_investment:.2f}"),
        html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")]
        + [html.Li(metric) for metric in stock_metrics]
    )

    # Cerates a dropdown for removing stocks from portfolio 
    options = [{"label": stock["symbol"], "value": stock["symbol"]} for stock in portfolio]

    # Creates a pie chart of stock distribution
    stock_distribution = px.pie(
        pd.DataFrame(portfolio), names="symbol", values="quantity", title="Stock Distribution"
    )

    return html.Ul([html.Li(item) for item in portfolio_list]), portfolio_metrics, options, stock_distribution


# Combined callback to manage crypto portfolio and display metrics
@app.callback(
    [Output("crypto-portfolio-display", "children"), Output("crypto-portfolio-metrics", "children"),
    Output("crypto-portfolio-remove-dropdown", "options"), Output("crypto-distribution", "figure")],
    [Input("add-to-crypto-portfolio-button", "n_clicks"), Input("remove-from-crypto-portfolio-button", "n_clicks")],
    [State("portfolio-crypto-input", "value"), State("crypto-portfolio-date-picker", "date"),
    State("crypto-portfolio-quantity-input", "value"), State("crypto-portfolio-price-input", "value"),
    State("crypto-portfolio-remove-dropdown", "value")])

# Define function that allows user to manage/edit crypto portfolio
def manage_crypto_portfolio(
    add_clicks, remove_clicks, crypto_name, purchase_date, quantity, price, remove_crypto):
    
    ctx = dash.callback_context
    if not ctx.triggered:
        return display_crypto_portfolio_metrics(crypto_portfolio)

    button_id = ctx.triggered[0]["prop_id"].split(".")[0]

    # Adds new crypto to portfolio
    if button_id == "add-to-crypto-portfolio-button" and crypto_name and purchase_date and quantity and price:
        new_entry = {"symbol": crypto_name, "date": purchase_date, "quantity": quantity, "price": price}
        crypto_portfolio.append(new_entry)

    # removes crypto from portfolio
    elif button_id == "remove-from-crypto-portfolio-button" and remove_crypto:
        crypto_portfolio[:] = [crypto for crypto in crypto_portfolio if crypto["symbol"] != remove_crypto]

    data["crypto_portfolio"] = crypto_portfolio
    write_data(DATA_FILE, data)

    return display_crypto_portfolio_metrics(crypto_portfolio)

# Define function to display crypto portfolio metrics
def display_crypto_portfolio_metrics(crypto_portfolio):
    # creates a list of crypto portfolio items
    crypto_portfolio_list = [
        f"{crypto['symbol']} - {crypto['quantity']} units bought on {crypto['date']} at ${crypto['price']}"
        for crypto in crypto_portfolio]

    total_value = 0
    total_quantity = 0
    total_investment = 0
    crypto_metrics = []

    # Creates total value, investment, and profit/loss for crypto portfolio 
    for crypto in crypto_portfolio:
        crypto_data = cryptocurrencies[cryptocurrencies["crypto_name"] == crypto["symbol"]]
        if not crypto_data.empty:
            current_price = crypto_data["price"].iloc[-1]
            investment_value = crypto["quantity"] * current_price
            total_investment += crypto["quantity"] * crypto["price"]
            profit_loss = investment_value - (crypto["quantity"] * crypto["price"])
            total_value += investment_value
            total_quantity += crypto["quantity"]
            crypto_metrics.append(
                f"""{crypto['symbol']}: {crypto['quantity']} units, Bought at ${crypto['price']}, \
                Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, \
                P/L: ${profit_loss:.2f}"""
            )

    # Creates list of important metrics for crypto portfolio 
    crypto_portfolio_metrics = html.Ul(
        [html.Li(f"Total Crypto Portfolio Value: ${total_value:.2f}"),
        html.Li(f"Total Units Owned: {total_quantity}"),
        html.Li(f"Total Investment: ${total_investment:.2f}"),
        html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")]
        + [html.Li(metric) for metric in crypto_metrics]
    )

    # Creates dropdown options for removing cryptos from portfolio
    crypto_options = [{"label": crypto["symbol"], "value": crypto["symbol"]} for crypto in crypto_portfolio]

    # Creates a pie chart
    crypto_distribution = px.pie(
        pd.DataFrame(crypto_portfolio), names="symbol", values="quantity", title="Crypto Distribution")

    return (
        html.Ul([html.Li(item) for item in crypto_portfolio_list]),
        crypto_portfolio_metrics,
        crypto_options,
        crypto_distribution)

# Runs the dash app
if __name__ == "__main__":
    app.run_server(debug=True)